# Phase 2: Silver Layer - Data Cleaning & Validation

This notebook cleans and validates data from the Bronze layer.

## Import Libraries

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, regexp_replace, trim, when, isnan, isnull, 
    split, element_at, length, upper, lower, 
    current_timestamp, lit, count, mean, median, 
    first, last, collect_list, size, array_contains
)
from pyspark.sql.types import *
from pyspark.sql.window import Window
import re
from datetime import datetime

## Configuration

In [0]:
# Source and target tables
BRONZE_TABLE_NAME = "bronze_vehicles"
SILVER_TABLE_NAME = "silver_vehicles"

# Outlier thresholds
MIN_YEAR = 1990
MAX_YEAR = datetime.now().year + 1
MIN_PRICE = 1000
MAX_PRICE = 1000000
MIN_KILOMETRES = 0
MAX_KILOMETRES = 500000

## Read Bronze Data

In [0]:
df_bronze = spark.table(BRONZE_TABLE_NAME)
print(f"Bronze data loaded: {df_bronze.count()} rows")
df_bronze.printSchema()

Bronze data loaded: 24198 rows
root
 |-- record_id: long (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)
 |-- source_file: string (nullable = true)
 |-- _c0: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Kilometres: string (nullable = true)
 |-- Body_Type: string (nullable = true)
 |-- Engine: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Drivetrain: string (nullable = true)
 |-- Exterior_Colour: string (nullable = true)
 |-- Interior_Colour: string (nullable = true)
 |-- Passengers: double (nullable = true)
 |-- Doors: string (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Highway: string (nullable = true)
 |-- Price: integer (nullable = true)



## Step 1: Extract Numeric Values from Kilometres

In [0]:
# Extract numeric value from "53052 km" format
df_cleaned = df_bronze.withColumn(
    "kilometres_numeric",
    regexp_replace(col("Kilometres"), r"[^0-9]", "").cast("int")
)

# Handle cases where extraction fails
df_cleaned = df_cleaned.withColumn(
    "kilometres_numeric",
    when(col("kilometres_numeric").isNull() | (col("kilometres_numeric") == 0), None)
    .otherwise(col("kilometres_numeric"))
)

print("Kilometres extraction sample:")
display(df_cleaned.select("Kilometres", "kilometres_numeric").limit(10))

Kilometres extraction sample:


Kilometres,kilometres_numeric
53052 km,53052
77127 km,77127
33032 km,33032
50702 km,50702
67950 km,67950
31000 km,31000
27800 km,27800
34396 km,34396
60892 km,60892
123535 km,123535


## Step 2: Extract Numeric Values from City/Highway Fuel Efficiency

In [0]:
# Function to extract numeric from fuel efficiency strings like "12.2L/100km" or "9.0L - 9.5L/100km"
def extract_fuel_efficiency(fuel_col):
    # Extract first number before "L" or range (take average of range)
    return when(
        col(fuel_col).isNotNull(),
        regexp_replace(
            regexp_replace(
                regexp_replace(col(fuel_col), r"L/100km", ""),
                r" - .*", ""  # Remove range part, keep first value
            ),
            r"[^0-9.]", ""
        ).cast("float")
    ).otherwise(None)

df_cleaned = df_cleaned \
    .withColumn("city_mpg_numeric", extract_fuel_efficiency("City")) \
    .withColumn("highway_mpg_numeric", extract_fuel_efficiency("Highway"))

print("Fuel efficiency extraction sample:")
display(df_cleaned.select("City", "city_mpg_numeric", "Highway", "highway_mpg_numeric").limit(10))

Fuel efficiency extraction sample:


City,city_mpg_numeric,Highway,highway_mpg_numeric
12.2L/100km,12.2,9.0L - 9.5L/100km,9.0
12.6L/100km,12.6,9.0L/100km,9.0
11.0L/100km,11.0,8.6L/100km,8.6
11.0L/100km,11.0,8.6L/100km,8.6
11.3L/100km,11.3,9.1L/100km,9.1
12.6L/100km,12.6,9.4L/100km,9.4
11.3L/100km,11.3,9.1L/100km,9.1
11.4L/100km,11.4,7.7L/100km,7.7
11.0L/100km,11.0,8.6L/100km,8.6
10.5L/100km,10.5,7.0L/100km,7.0


## Step 3: Data Type Conversions

In [0]:
# Convert Year to integer
df_cleaned = df_cleaned.withColumn(
    "year_int",
    col("Year").cast("int")
)

# Price is already numeric, but ensure it's float
df_cleaned = df_cleaned.withColumn(
    "price_float",
    col("Price").cast("float")
)

print("Data type conversions sample:")
display(df_cleaned.select("Year", "year_int", "Price", "price_float").limit(5))

Data type conversions sample:


Year,year_int,Price,price_float
2019,2019,43880,43880.0
2018,2018,36486,36486.0
2019,2019,40888,40888.0
2020,2020,44599,44599.0
2021,2021,46989,46989.0


## Step 4: Standardize Categorical Values

In [0]:
# Standardize Fuel Type: "Gas" -> "Gasoline"
df_cleaned = df_cleaned.withColumn(
    "fuel_type_standardized",
    when(upper(col("Fuel_Type")).contains("GASOLINE"), "Gasoline")
    .when(upper(col("Fuel_Type")).contains("GAS"), "Gasoline")
    .when(upper(col("Fuel_Type")).contains("PREMIUM"), "Premium Unleaded")
    .when(upper(col("Fuel_Type")).contains("DIESEL"), "Diesel")
    .when(upper(col("Fuel_Type")).contains("ELECTRIC"), "Electric")
    .when(upper(col("Fuel_Type")).contains("HYBRID"), "Hybrid")
    .otherwise(trim(upper(col("Fuel_Type"))))
)

# Standardize Transmission (remove extra spaces, standardize case)
df_cleaned = df_cleaned.withColumn(
    "transmission_standardized",
    trim(upper(col("Transmission")))
)

# Standardize Engine (extract key info - simplified)
df_cleaned = df_cleaned.withColumn(
    "engine_standardized",
    trim(upper(col("Engine")))
)

print("Categorical standardization sample:")
display(df_cleaned.select("Fuel_Type", "fuel_type_standardized", 
                  "Transmission", "transmission_standardized").limit(10))

Categorical standardization sample:


Fuel_Type,fuel_type_standardized,Transmission,transmission_standardized
Gas,Gasoline,9 Speed Automatic,9 SPEED AUTOMATIC
Gas,Gasoline,9 Speed Automatic,9 SPEED AUTOMATIC
Premium Unleaded,Premium Unleaded,10 Speed Automatic,10 SPEED AUTOMATIC
Gas,Gasoline,null,null
Gas,Gasoline,null,null
Gas,Gasoline,Automatic,AUTOMATIC
Premium Unleaded,Premium Unleaded,10 Speed Automatic,10 SPEED AUTOMATIC
Premium Unleaded,Premium Unleaded,9 Speed Automatic,9 SPEED AUTOMATIC
Gas,Gasoline,Automatic,AUTOMATIC
Gas,Gasoline,6 Speed Manual,6 SPEED MANUAL


## Step 5: Handle Missing Values - Kilometres

In [0]:
# Calculate median kilometres by Make/Model/Year group
window_spec = Window.partitionBy("Make", "Model", "year_int")

# Calculate median for each group
df_with_median = df_cleaned.withColumn(
    "median_km_by_group",
    median("kilometres_numeric").over(window_spec)
)

# Impute missing kilometres with group median, fallback to overall median
overall_median_km = df_cleaned.agg(median("kilometres_numeric").alias("median")).collect()[0]["median"]

df_cleaned = df_with_median.withColumn(
    "kilometres_imputed",
    when(col("kilometres_numeric").isNull(), 
         when(col("median_km_by_group").isNotNull(), col("median_km_by_group"))
         .otherwise(overall_median_km))
    .otherwise(col("kilometres_numeric"))
)

print(f"Overall median kilometres: {overall_median_km}")
print("Kilometres imputation sample:")
display(df_cleaned.select("Make", "Model", "year_int", "kilometres_numeric", 
                  "kilometres_imputed").filter(col("kilometres_numeric").isNull()).orderBy(F.rand()).limit(5))

Overall median kilometres: 57187.0
Kilometres imputation sample:


Make,Model,year_int,kilometres_numeric,kilometres_imputed
GMC,Sierra,2023,null,10.0
Chrysler,Grand,2023,null,57187.0
Jeep,Wrangler,2023,null,16.0
Jeep,Compass,2022,null,144.0
Nissan,Rogue,2023,null,90.0


## Step 6: Handle Missing Values - Engine/Transmission

In [0]:
# Impute Engine and Transmission with mode by Make/Model
window_make_model = Window.partitionBy("Make", "Model")

# Get mode (most frequent) for Engine
df_with_engine_mode = df_cleaned.withColumn(
    "engine_mode",
    first("engine_standardized", ignorenulls=True).over(window_make_model)
)

df_cleaned = df_with_engine_mode.withColumn(
    "engine_imputed",
    when(col("engine_standardized").isNull() | (col("engine_standardized") == ""), 
         col("engine_mode"))
    .otherwise(col("engine_standardized"))
)

# Get mode for Transmission
df_with_trans_mode = df_cleaned.withColumn(
    "transmission_mode",
    first("transmission_standardized", ignorenulls=True).over(window_make_model)
)

df_cleaned = df_with_trans_mode.withColumn(
    "transmission_imputed",
    when(col("transmission_standardized").isNull() | (col("transmission_standardized") == ""), 
         col("transmission_mode"))
    .otherwise(col("transmission_standardized"))
)

print("Engine/Transmission imputation sample:")
display(df_cleaned.select("Make", "Model", "Engine", "engine_imputed", "Transmission", "transmission_imputed")
        .orderBy(F.rand()).limit(10))

Engine/Transmission imputation sample:


Make,Model,Engine,engine_imputed,Transmission,transmission_imputed
Acura,RDX,3.5,3.5,6 Speed Automatic,6 SPEED AUTOMATIC
Aston Martin,Martin,null,8 CYLINDER ENGINE,Automatic,AUTOMATIC
Volvo,XC90,4 Cylinder Engine,4 CYLINDER ENGINE,Automatic,AUTOMATIC
Mazda,CX-5,2.5L 4cyl,2.5L 4CYL,6 Speed Automatic,6 SPEED AUTOMATIC
Porsche,Panamera,8 Cylinder,8 CYLINDER,8 Speed Automatic,8 SPEED AUTOMATIC
Volkswagen,Beetle,4 Cylinder Engine,4 CYLINDER ENGINE,6 Speed Automatic,6 SPEED AUTOMATIC
Jeep,Gladiator,3.6L Pentastar VVT V6 w/ESS,3.6L PENTASTAR VVT V6 W/ESS,null,AUTOMATIC
Kia,K5,null,4 CYLINDER ENGINE,Automatic,AUTOMATIC
Ford,Explorer,null,3.5L 6CYL,Automatic,AUTOMATIC
Nissan,Rogue,4 Cylinder Engine,4 CYLINDER ENGINE,CVT,CVT


## Step 7: Handle Missing Values - City/Highway MPG

In [0]:
# Calculate median by Make/Model for City and Highway
window_make_model_mpg = Window.partitionBy("Make", "Model")

df_with_mpg_median = df_cleaned.withColumn(
    "median_city_by_group",
    median("city_mpg_numeric").over(window_make_model_mpg)
).withColumn(
    "median_highway_by_group",
    median("highway_mpg_numeric").over(window_make_model_mpg)
)

# Overall medians
overall_median_city = df_cleaned.agg(median("city_mpg_numeric").alias("median")).collect()[0]["median"]
overall_median_highway = df_cleaned.agg(median("highway_mpg_numeric").alias("median")).collect()[0]["median"]

# Impute: if both missing, keep as null (will drop later), else impute with group median or overall median
df_cleaned = df_with_mpg_median.withColumn(
    "city_mpg_imputed",
    when(col("city_mpg_numeric").isNull(),
         when(col("median_city_by_group").isNotNull(), col("median_city_by_group"))
         .otherwise(overall_median_city))
    .otherwise(col("city_mpg_numeric"))
).withColumn(
    "highway_mpg_imputed",
    when(col("highway_mpg_numeric").isNull(),
         when(col("median_highway_by_group").isNotNull(), col("median_highway_by_group"))
         .otherwise(overall_median_highway))
    .otherwise(col("highway_mpg_numeric"))
)

print(f"Overall median City MPG: {overall_median_city}")
print(f"Overall median Highway MPG: {overall_median_highway}")

Overall median City MPG: 11.100000381469727
Overall median Highway MPG: 8.399999618530273


## Step 8: Handle Missing Values - Passengers/Doors

Impute based on Body Type defaults

In [0]:
# Define defaults by Body Type
body_type_defaults = {
    "SUV": {"passengers": 5.0, "doors": 4.0},
    "Sedan": {"passengers": 5.0, "doors": 4.0},
    "Truck": {"passengers": 2.0, "doors": 2.0},
    "Coupe": {"passengers": 4.0, "doors": 2.0},
    "Wagon": {"passengers": 5.0, "doors": 4.0},
    "Hatchback": {"passengers": 5.0, "doors": 4.0}
}

# Impute Passengers
df_cleaned = df_cleaned.withColumn(
    "passengers_imputed",
    when(col("Passengers").isNull() | isnan(col("Passengers")),
         when(upper(col("Body_Type")).contains("SUV"), 5.0)
         .when(upper(col("Body_Type")).contains("TRUCK"), 2.0)
         .when(upper(col("Body_Type")).contains("COUPE"), 4.0)
         .otherwise(5.0))
    .otherwise(col("Passengers").cast("float"))
)

# Impute Doors
df_cleaned = df_cleaned.withColumn(
    "doors_imputed",
    when(col("Doors").isNull() | (col("Doors") == ""),
         when(upper(col("Body_Type")).contains("COUPE"), 2.0)
         .when(upper(col("Body_Type")).contains("TRUCK"), 2.0)
         .otherwise(4.0))
    .otherwise(
        when(col("Doors").rlike(r"\d+"), 
             regexp_replace(col("Doors"), r"[^0-9]", "").cast("float"))
        .otherwise(4.0)
    )
)

## Step 9: Outlier Removal

In [0]:
# Filter outliers
initial_count = df_cleaned.count()

df_cleaned = df_cleaned.filter(
    (col("year_int") >= MIN_YEAR) & 
    (col("year_int") <= MAX_YEAR) &
    (col("price_float") >= MIN_PRICE) & 
    (col("price_float") <= MAX_PRICE) &
    (col("kilometres_imputed") >= MIN_KILOMETRES) & 
    (col("kilometres_imputed") <= MAX_KILOMETRES)
)

final_count = df_cleaned.count()
removed_count = initial_count - final_count

print(f"Initial rows: {initial_count}")
print(f"Rows after outlier removal: {final_count}")
print(f"Rows removed: {removed_count} ({round(removed_count/initial_count*100, 2)}%)")

Initial rows: 24198
Rows after outlier removal: 24071
Rows removed: 127 (0.52%)


## Step 10: Remove Duplicates

In [0]:
# Remove exact duplicates based on key fields
before_dedup = df_cleaned.count()

# Key fields for duplicate detection
key_fields = ["Make", "Model", "year_int", "kilometres_imputed", "price_float", 
              "Body_Type", "engine_imputed", "transmission_imputed"]

df_cleaned = df_cleaned.dropDuplicates(key_fields)

after_dedup = df_cleaned.count()
dup_count = before_dedup - after_dedup

print(f"Rows before deduplication: {before_dedup}")
print(f"Rows after deduplication: {after_dedup}")
print(f"Duplicates removed: {dup_count}")

Rows before deduplication: 24071
Rows after deduplication: 19461
Duplicates removed: 4610


## Step 11: Create Silver Table Schema

In [0]:
# Select and rename columns for Silver layer
df_silver = df_cleaned.select(
    col("record_id"),
    col("ingestion_timestamp"),
    col("source_file"),
    col("Make"),
    col("Model"),
    col("year_int").alias("Year"),
    col("kilometres_imputed").alias("Kilometres"),
    col("Body_Type").alias("BodyType"),
    col("engine_imputed").alias("Engine"),
    col("transmission_imputed").alias("Transmission"),
    col("Drivetrain"),
    col("Exterior_Colour").alias("ExteriorColour"),
    col("Interior_Colour").alias("InteriorColour"),
    col("passengers_imputed").alias("Passengers"),
    col("doors_imputed").alias("Doors"),
    col("fuel_type_standardized").alias("FuelType"),
    col("city_mpg_imputed").alias("City"),
    col("highway_mpg_imputed").alias("Highway"),
    col("price_float").alias("Price"),
    current_timestamp().alias("silver_processing_timestamp")
)

# Drop rows where both City and Highway are still null (couldn't be imputed)
df_silver = df_silver.filter(
    col("City").isNotNull() | col("Highway").isNotNull()
)

print("Silver schema:")
df_silver.printSchema()
print(f"\nSilver row count: {df_silver.count()}")

Silver schema:
root
 |-- record_id: long (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)
 |-- source_file: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Kilometres: double (nullable = true)
 |-- BodyType: string (nullable = true)
 |-- Engine: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Drivetrain: string (nullable = true)
 |-- ExteriorColour: string (nullable = true)
 |-- InteriorColour: string (nullable = true)
 |-- Passengers: double (nullable = true)
 |-- Doors: double (nullable = true)
 |-- FuelType: string (nullable = true)
 |-- City: double (nullable = true)
 |-- Highway: double (nullable = true)
 |-- Price: float (nullable = true)
 |-- silver_processing_timestamp: timestamp (nullable = false)


Silver row count: 19461


## Step 12: Write to Delta Lake

In [0]:
# Write to Delta table (managed table in Unity Catalog)
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(SILVER_TABLE_NAME)

print(f"Silver table '{SILVER_TABLE_NAME}' created successfully!")
print(f"Table is managed by Unity Catalog")

# Verify table creation
print("\nVerification - Sample from Silver table:")
display(spark.table(SILVER_TABLE_NAME).orderBy(F.rand()).limit(10))

print(f"\nFinal row count: {spark.table(SILVER_TABLE_NAME).count()}")

Silver table 'silver_vehicles' created successfully!
Table is managed by Unity Catalog

Verification - Sample from Silver table:


record_id,ingestion_timestamp,source_file,Make,Model,Year,Kilometres,BodyType,Engine,Transmission,Drivetrain,ExteriorColour,InteriorColour,Passengers,Doors,FuelType,City,Highway,Price,silver_processing_timestamp
17071,2025-12-05T04:03:04.121Z,ensf612project-data.csv,Hyundai,Palisade,2021,76158.0,SUV,3.8L 6CYL,8 SPEED AUTOMATIC,AWD,Dark Grey,Dark Grey,8.0,5.0,Gasoline,12.300000190734863,9.600000381469727,44995.0,2025-12-05T19:31:27.965Z
10840,2025-12-05T04:03:04.121Z,ensf612project-data.csv,Dodge,Journey,2014,186000.0,SUV,V6 CYLINDER ENGINE,6 SPEED AUTOMATIC,FWD,White,Black,7.0,4.0,Gasoline,12.699999809265137,8.699999809265137,10800.0,2025-12-05T19:31:27.965Z
14053,2025-12-05T04:03:04.121Z,ensf612project-data.csv,GMC,Sierra,2023,276.0,Truck,6.2L 8CYL,10 SPEED AUTOMATIC,4x4,Dynamic Blue Metallic,Black,2.0,2.0,Premium Unleaded,15.399999618530273,12.300000190734863,86943.0,2025-12-05T19:31:27.965Z
18047,2025-12-05T04:03:04.121Z,ensf612project-data.csv,Infiniti,QX60,2019,63742.0,SUV,3.5L 6CYL,CVT,AWD,null,null,5.0,4.0,Gasoline,12.5,9.0,32588.0,2025-12-05T19:31:27.965Z
11045,2025-12-05T04:03:04.121Z,ensf612project-data.csv,Dodge,Journey,2019,117293.5,SUV,2.4L TIGERSHARK MULTIAIR VVT I-4 ENGINE,AUTOMATIC,FWD,Pitch Black,null,5.0,4.0,REGULAR UNLEADED,12.699999809265137,9.199999809265137,28460.0,2025-12-05T19:31:27.965Z
4086,2025-12-05T04:03:04.121Z,ensf612project-data.csv,Cadillac,XT5,2021,38059.0,SUV,3.6L 6CYL,9 SPEED AUTOMATIC,AWD,Stellar Black Metallic,Black,5.0,4.0,Gasoline,12.899999618530273,9.199999809265137,46488.0,2025-12-05T19:31:27.965Z
6452,2025-12-05T04:03:04.121Z,ensf612project-data.csv,Porsche,911,2021,3413.0,Coupe,6 CYLINDER,AUTOMATIC,AWD,Dolomite Silver Metallic,null,4.0,2.0,Gasoline,13.100000381469727,9.800000190734863,314800.0,2025-12-05T19:31:27.965Z
17039,2025-12-05T04:03:04.121Z,ensf612project-data.csv,Hyundai,Tucson,2021,23133.0,SUV,4 CYLINDER ENGINE,6 SPEED AUTOMATIC,FWD,White,Black,5.0,4.0,Gasoline,10.0,7.900000095367432,28999.0,2025-12-05T19:31:27.965Z
22373,2025-12-05T04:03:04.121Z,ensf612project-data.csv,Nissan,Sentra,2013,140000.0,Sedan,4 CYL,AUTOMATIC,FWD,White,null,5.0,4.0,Gasoline,8.0,6.0,8999.0,2025-12-05T19:31:27.965Z
17993,2025-12-05T04:03:04.121Z,ensf612project-data.csv,Infiniti,Q50,2019,39000.0,Sedan,V6 CYLINDER ENGINE,AUTOMATIC,AWD,Black Obsidian,Black,5.0,4.0,Gasoline,12.399999618530273,8.699999809265137,33950.0,2025-12-05T19:31:27.965Z



Final row count: 19461


## Data Quality Summary

In [0]:
print("=" * 80)
print("SILVER LAYER CLEANING COMPLETE")
print("=" * 80)
print(f"Source: {BRONZE_TABLE_NAME}")
print(f"Target: {SILVER_TABLE_NAME}")
print(f"Initial rows: {initial_count}")
print(f"Final rows: {final_count}")
print(f"Rows removed (outliers): {removed_count}")
print(f"Duplicates removed: {dup_count}")
print(f"Processing timestamp: {datetime.now()}")
print("=" * 80)

# Show data quality metrics
print("\nData Quality Metrics:")
quality_metrics = df_silver.agg(
    count("*").alias("total_rows"),
    count(when(col("Kilometres").isNull(), 1)).alias("null_kilometres"),
    count(when(col("Engine").isNull() | (col("Engine") == ""), 1)).alias("null_engine"),
    count(when(col("Transmission").isNull() | (col("Transmission") == ""), 1)).alias("null_transmission"),
    count(when(col("City").isNull(), 1)).alias("null_city"),
    count(when(col("Highway").isNull(), 1)).alias("null_highway")
)
display(quality_metrics)


SILVER LAYER CLEANING COMPLETE
Source: bronze_vehicles
Target: silver_vehicles
Initial rows: 24198
Final rows: 24071
Rows removed (outliers): 127
Duplicates removed: 4610
Processing timestamp: 2025-12-05 04:27:39.909286

Data Quality Metrics:


total_rows,null_kilometres,null_engine,null_transmission,null_city,null_highway
19461,0,99,34,0,0
